In [414]:
import names
from syft.core.common import UID
from sympy import symbols

def create_lookup_tables_for_symbol(polynomial):

    index2symbol = [str(x) for x in polynomial.free_symbols]
    symbol2index = {sym: i for i, sym in enumerate(index2symbol)}

    return index2symbol, symbol2index

def create_searchable_function(f, symbol2index):

        # Tudor: Here you weren't using *params
        # Tudor: If I understand correctly, .subs returns
        def _run_specific_args(tuple_of_args: tuple) -> Any:
            kwargs = {sym: tuple_of_args[i] for sym, i in symbol2index.items()}
            output =  f.subs(kwargs)
            return output

        return _run_specific_args

def minimize_function(f, rranges):
    
    # Step 1: try simplicial
    shgo_results = optimize.shgo(f, rranges, sampling_method='simplicial')
    max_val,results = -float(shgo_results.fun), shgo_results

    if not results.success:
        # sometimes simplicial has trouble as a result of initialization
        # see: https://github.com/scipy/scipy/issues/10429 for details
        print("Simplicial search didn't work... trying sobol")
        shgo_results = optimize.shgo(f, rranges, sampling_method='sobol')
        L,results = -float(shgo_results.fun), shgo_results
        
    return results
    
class Entity():
    
    def __init__(self, unique_name:str=None, id=None):
        
        # If someone doesn't provide a unique name - make one up!
        if unique_name is None:
            unique_name = names.get_full_name().replace(" ","_")
        self.unique_name = unique_name
            
        self.id = id if id else UID()
            
    def __repr__(self):
        return "<Entity:"+self.unique_name+">"
    
# scalar_name2obj is used to look and extract value, min_val and max_val
# if we are able to store these in the name string instead we could extract them
# at the point of use instead of lookup
ssid2obj = {}

class Scalar():
    ""

class IntermediateScalar(Scalar):
    
    def __init__(self, poly):
        self.poly = poly
        
    def __str__(self) -> str:
        return str(self.poly)

    def __repr__(self) -> str:
        return str(self)
    
    def __rmul__(self, other: "Scalar") -> "Scalar":
        return self * other

    def __radd__(self, other: "Scalar") -> "Scalar":
        return self + other
    
    @property
    def input_scalars(self):
        phi_scalars = list()
        for ssid in self.poly.free_symbols:
            phi_scalars.append(ssid2obj[str(ssid)])
        return phi_scalars
    
    @property
    def max_val(self):
        i2s,s2i = create_lookup_tables_for_symbol(self.poly)
        f = create_searchable_function(f=-self.poly, symbol2index=s2i)

        rranges = [(x.min_val, x.max_val) for x in z.input_scalars]
        
        return -minimize_function(f,rranges).fun
    
    @property
    def min_val(self):
        i2s,s2i = create_lookup_tables_for_symbol(self.poly)
        f = create_searchable_function(f=self.poly, symbol2index=s2i)

        rranges = [(x.min_val, x.max_val) for x in z.input_scalars]
        
        return minimize_function(f,rranges).fun

    @property
    def value(self):
        return self.poly.subs({obj.poly:obj.value for obj in self.input_scalars})
    
class IntermediatePhiScalar(IntermediateScalar):
    
    
    def to_gamma_scalars(self, *args):
        
        new_self = GammaScalar(min_val=self.min_val,
                               value=self.value,
                               max_val=self.max_val,
                               entity=self.entity)
        
        new_args = list()
        for arg in args:
            
            new_arg = GammaScalar(min_val=arg.min_val,
                                  value=arg.value,
                                  max_val=arg.max_val,
                                  entity=arg.entity)
        
            
            new_args.append(new_arg)
        
        return new_self, new_args
    

    def __mul__(self, other: "Scalar") -> "Scalar":

        if isinstance(other, IntermediatePhiScalar):
            return IntermediatePhiScalar(self.poly * other.poly)
        
        # if other is referencing the same individual
        if self.entity == other.entity:
            return IntermediatePhiScalar(self.poly + other.poly)

        new_self, new_args = self.to_gamma_scalars(other)

        return new_self * new_args[0]

    def __add__(self, other: "Scalar") -> "Scalar":

        # if other is a public value
        if not isinstance(other, IntermediatePhiScalar):
            return IntermediatePhiScalar(self.poly + other)

        # if other is referencing the same individual
        if self.entity == other.entity:
            return IntermediatePhiScalar(self.poly + other.poly)
        
        new_self, new_args = self.to_gamma_scalars(other)

        return new_self + new_args[0]
    
    
    def __sub__(self, other: "Scalar") -> "Scalar":

        # if other is a public value
        if not isinstance(other, IntermediatePhiScalar):
            return IntermediatePhiScalar(self.poly - other)

        # if other is referencing the same individual
        if self.entity == other.entity:
            return IntermediatePhiScalar(self.poly - other.poly)
        
        new_self, new_args = self.to_gamma_scalars(other)

        return new_self - new_args[0]    

    def __sub__(self, other: "IntermediatePhiScalar") -> "IntermediatePhiScalar":
        if isinstance(other, IntermediatePhiScalar):
            return IntermediatePhiScalar(self.poly - other.poly)
        
        return IntermediatePhiScalar(self.poly - other)
    
class OriginScalar(Scalar):
    
    def __init__(self, min_val, value, max_val, entity=None, id=None):
        
        self.id = id if id else UID()
        self._value = value
        self._min_val = min_val
        self._max_val = max_val
        self.entity = entity if entity is not None else Entity()

    @property
    def value(self):
        return self._value
        
    @property
    def max_val(self):
        return self._max_val
    
    @property
    def min_val(self):
        return self._min_val
    
class PhiScalar(OriginScalar, IntermediatePhiScalar):
    """A scalar over data from a single entity"""
    
    def __init__(self, min_val, value, max_val, entity=None, id=None, ssid=None):
        super().__init__(min_val=min_val, value=value, max_val=max_val, entity=entity,id=id)
        
        # the scalar string identifier (SSID) - because we're using polynomial libraries
        # we need to be able to reference this object in string form. the library doesn't
        # know how to process things that aren't strings
        if ssid is None:
            ssid = str(self.id).split(" ")[1][:-1]# + "_" + str(self.entity.id).split(" ")[1][:-1]
            
        self.ssid = ssid
        
        IntermediatePhiScalar.__init__(self, poly=symbols(self.ssid))
        
        ssid2obj[self.ssid] = self
    
    
class IntermediateGammaScalar(IntermediateScalar):
    """"""
    
    def __add__(self, other):
        if isinstance(other, Scalar):
            return IntermediateGammaScalar(poly=self.poly + other.poly)
        return IntermediateGammaScalar(poly=self.poly + other)
    
    def __sub__(self, other):
        if isinstance(other, Scalar):
            return IntermediateGammaScalar(poly=self.poly - other.poly)
        return IntermediateGammaScalar(poly=self.poly - other)    
    
    def __mult__(self, other):
        if isinstance(other, Scalar):
            return IntermediateGammaScalar(poly=self.poly * other.poly)
        return IntermediateGammaScalar(poly=self.poly * other)    
        
        
class GammaScalar(OriginScalar, IntermediateGammaScalar):
    """A scalar over data from multiple entities"""
    
    def __init__(self, min_val, value, max_val, entity=None, id=None, ssid=None):
        super().__init__(min_val=min_val, value=value, max_val=max_val, entity=entity, id=id)
        
        # the scalar string identifier (SSID) - because we're using polynomial libraries
        # we need to be able to reference this object in string form. the library doesn't
        # know how to process things that aren't strings
        if ssid is None:
            ssid = str(self.id).split(" ")[1][:-1] + "_" + str(self.entity.id).split(" ")[1][:-1]
            
        self.ssid = ssid
        
        IntermediateGammaScalar.__init__(self, poly=symbols(self.ssid))
        
        ssid2obj[self.ssid] = self
    
        


In [415]:
e = Entity()
x = PhiScalar(0,1,2,ssid='x')
y = PhiScalar(0,1.5,3,ssid='y')
z = x + y

In [416]:
o = z + z + 3

In [417]:
o

2*87ea181ad0b64edcaf430f70d394a5fc_c165bf28cc9d48c39c1b89386879aea9 + 2*ea863d260eaf478b90f6be35c2ee585e_f201375a07b44904a696e0d8c6b80b55 + 3

In [418]:
ssid2obj

{'x': x,
 'y': y,
 'ea863d260eaf478b90f6be35c2ee585e_f201375a07b44904a696e0d8c6b80b55': ea863d260eaf478b90f6be35c2ee585e_f201375a07b44904a696e0d8c6b80b55,
 '87ea181ad0b64edcaf430f70d394a5fc_c165bf28cc9d48c39c1b89386879aea9': 87ea181ad0b64edcaf430f70d394a5fc_c165bf28cc9d48c39c1b89386879aea9}

In [27]:
import numpy as np
import sympy as sym
from sympy.abc import *
from scipy import optimize

def f(x, y):
    return x**2 + y**2

out = f(x,y)

out = sym.Matrix([out])
j = out.jacobian([x,y])
K = -np.sum(np.square(j))**0.5

K_i2s, K_s2i = create_lookup_tables_for_symbol(K)
search_fun = create_run_specific_args(K, K_s2i)
print(K_i2s)

rranges = list()
rranges.append((-5,5)) # x
rranges.append((-5,5)) # y



# Step 1: try simplicial
shgo_results = optimize.shgo(search_fun, rranges, sampling_method='simplicial')
L,results = -float(shgo_results.fun), shgo_results

if not results.success:
    # sometimes simplicial has trouble as a result of initialization
    # see: https://github.com/scipy/scipy/issues/10429 for details
    print("Simplicial search didn't work... trying sobol")
    shgo_results = optimize.shgo(search_fun, rranges, sampling_method='sobol')
    L,results = -float(shgo_results.fun), shgo_results
    
if not results.success:
    print("Derivative approach seems to be having trouble... trying Lipschitz search...")
    r1 = np.array([x, y])
    r2 = np.array([x+a, y+b])

    left = np.sum(np.square(f(*r1) - f(*r2)))**0.5
    right = np.sum(np.square(r1 - r2))**0.5

    K = -left / right
    K

    K_i2s, K_s2i = create_lookup_tables_for_symbol(K)
    search_fun = create_run_specific_args(K, K_s2i)

    rranges = list()
    rranges.append((-5,5)) # a
    rranges.append((-5,5)) # b
    rranges.append((-5,5)) # x
    rranges.append((-5,5)) # y


    constraints = list()
    constraints.append({'type':'ineq', 'fun':lambda x: x[2] + x[0] + 5}) # x + a + 5 >= 0.     x + a >= -5
    constraints.append({'type':'ineq', 'fun':lambda x: -x[2] - x[0] + 5}) # -x -a + 5 >= 0     
    constraints.append({'type':'ineq', 'fun':lambda x: x[3] + x[1] + 5})
    constraints.append({'type':'ineq', 'fun':lambda x: -x[3] - x[1] + 5})
    constraints.append({'type':'ineq', 'fun':lambda x: np.sqrt(x[0]**2 + x[1]**2) - np.finfo(float).eps})

    shgo_results = optimize.shgo(search_fun, rranges, sampling_method='simplicial', constraints=constraints)
    L,results = -float(shgo_results.fun), shgo_results
    
if not results.success:
    # sometimes simplicial has trouble as a result of initialization
    # see: https://github.com/scipy/scipy/issues/10429 for details
    print("Simplicial search didn't work... trying sobol")
    shgo_results = optimize.shgo(search_fun, rranges, sampling_method='sobol')

    L,results = -float(shgo_results.fun), shgo_results

['x', 'y']
Simplicial search didn't work... trying sobol
